In [1]:
!git clone https://github.com/YerevaNN/mimic3-benchmarks/

Cloning into 'mimic3-benchmarks'...
remote: Enumerating objects: 1716, done.
remote: Total 1716 (delta 0), reused 0 (delta 0), pack-reused 1716
Receiving objects: 100% (1716/1716), 11.24 MiB | 26.16 MiB/s, done.
Resolving deltas: 100% (1221/1221), done.
